# **Activate Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import shutil
import zipfile

zip_fold="/content/drive/MyDrive/schizo_dataset.zip"
shutil.copy(zip_fold,".")

with zipfile.ZipFile('./schizo_dataset.zip','r') as zip_ref:
  zip_ref.extractall('.')

In [ ]:
%%capture
pip install mne

# **Read Data & Preprocessing**

In [ ]:
from glob import glob
import os
import mne
import numpy as np
import pandas
import matplotlib.pyplot as plt


In [ ]:
dataset=glob('schizo_dataset/*.edf')
print(len(dataset))


28


In [ ]:
healty=[i for i in dataset if 'h' in i.split('/')[1]]
sick=[i for i in dataset if 's' in i.split('/')[1]]


In [ ]:
def read_data(file_path):
  data=mne.io.read_raw_edf(file_path,preload=True)
  data.set_eeg_reference()# reference to
  data.filter(l_freq=0.5,h_freq=45)#bandpass filter
  epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)#split signals 5 second epochs
  array=epochs.get_data()
  return array

In [ ]:
%%capture
healty_data=[read_data(i) for i in healty]
sick_data=[read_data(i) for i in sick]

In [ ]:
#Create Label
hlabel=[len(i)*[0] for i in healty_data]
slabel=[len(i)*[1] for i in sick_data]

data_list=healty_data+sick_data
label_list=hlabel+slabel

grouplist=[[i]*len(j) for i,j in enumerate(data_list)]

data_array=np.vstack(data_list)
label_array=np.hstack(label_list)
group_array=np.hstack(grouplist)

In [ ]:
print(data_array.shape,label_array.shape,group_array.shape)

(7201, 19, 1250) (7201,) (7201,)


In [ ]:
from scipy import stats
def mean(x):
  return np.mean(x, axis=-1)
def std(x):
  return np.std(x, axis=-1)
def ptp(x):
  return np.ptp(x, axis=-1)
def var(x):
  return np.var(x,axis=-1)
def minim(x):
  return np.min(x,axis=-1)
def maxim(x):
  return np.max(x,axis=-1)
def argminim(x):
  return np.argmin(x,axis=-1)
def argmaxim(x):
  return np.argmax(x,axis=-1)
def rms(x):
  return np.sqrt(np.mean(x**2,axis=-1))
def abs_diff_signal(x):
  return np.sum(np.abs(np.diff(x,axis=-1)),axis=-1)
def skewness(x):
  return stats.skew(x,axis=-1)
def kurtosis(x):
  return stats.kurtosis(x, axis=-1)
def concatenate_features(x):
  return np.concatenate((mean(x), std(x), ptp(x), var(x), minim(x), maxim(x), argminim(x),
                        argmaxim(x), rms(x), abs_diff_signal(x),skewness(x),kurtosis(x)), axis=-1 )

In [ ]:
features=[]
for d in data_array:
  features.append(concatenate_features(d))

features=np.array(features)


# **Model Training**

In [ ]:

"""    Chrono Net Deep Learning Model     """



import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import time

def chrono_net(input_shape):
    model = Sequential()

    model.add(Conv1D(64, 3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))

    model.add(LSTM(100, return_sequences=True))
    model.add(Dropout(0.3))

    model.add(LSTM(100, return_sequences=False))
    model.add(Dropout(0.3))


    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    return model


X = features
y = label_array

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

model = chrono_net(X_train.shape[1:])

history = model.fit(X_train, y_train, epochs=30, batch_size=16, validation_split=0.2, verbose=1)


train_accuracy = history.history['accuracy'][-1]
val_accuracy = history.history['val_accuracy'][-1]

print(f'Train Accuracy: {train_accuracy:.4f}')
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {test_acc:.4f}')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
288/288 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.5880 - loss: 0.6627 - val_accuracy: 0.5938 - val_loss: 0.6492
Epoch 2/30
288/288 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.6364 - loss: 0.6309 - val_accuracy: 0.6502 - val_loss: 0.6057
Epoch 3/30
288/288 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6540 - loss: 0.6131 - val_accuracy: 0.6250 - val_loss: 0.6154
Epoch 4/30
288/288 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.6685 - loss: 0.5939 - val_accuracy: 0.7031 - val_loss: 0.5613
Epoch 5/30
288/288 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7205 - loss: 0.5445 - val_accuracy: 0.7639 - val_loss: 0.5040
Epoch 6/30
288/288 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7368 - loss: 0.5206 - val_accuracy: 0.8073 - val_loss: 0.4314
Epoch 7/30
288/288 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.7494 - loss: 0.5014 - val_accuracy: 0.8247 - val_loss: 0.4086
Epoch 8/30
288/288 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7929 - loss: 0.4326 - val_acc

In [ ]:
import psutil
from tensorflow.keras import backend as K
param_count = model.count_params()
process = psutil.Process()
memory_usage = process.memory_info().rss / (1024 * 1024)

print(f"Memory Usage: {memory_usage:.2f} MB")

Memory Usage: 4575.78 MB


In [ ]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_12 (Conv1D)                   │ (None, 226, 64)             │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_12 (MaxPooling1D)      │ (None, 113, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_30 (Dropout)                 │ (None, 113, 64)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_13 (Conv1D)                   │ (None, 111, 128)            │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_13 (MaxPooling1D)      │ (None, 55, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_31 (Dropout)                 │ (None, 55, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_12 (LSTM)                       │ (None, 55, 100)             │          91,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_32 (Dropout)                 │ (None, 55, 100)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_13 (LSTM)                       │ (None, 100)                 │          80,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_33 (Dropout)                 │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 64)                  │           6,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_34 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 610,469 (2.33 MB)

 Trainable params: 203,489 (794.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 406,980 (1.55 MB)